In [1]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta, date
from calendar import monthrange
import random

In [3]:
# 1) Instanciar Faker para Brasil e fixar seeds (reprodutibilidade)
fake = Faker('pt_BR')
Faker.seed(42)
np.random.seed(42)
random.seed(42)

In [4]:
# 2) Configuração: poucas entradas
num_records = 8950

In [5]:
today_dt = datetime.now()
today_d = today_dt.date()

In [6]:
# Estados e cidades do Brasil (probabilidades aproximadas, concentrando em SP)
states_cities = {
    'São Paulo': (['São Paulo', 'Campinas', 'Santos', 'Sorocaba'], [0.60, 0.18, 0.12, 0.10]),
    'Rio de Janeiro': (['Rio de Janeiro', 'Niterói', 'Duque de Caxias'], [0.70, 0.18, 0.12]),
    'Minas Gerais': (['Belo Horizonte', 'Contagem', 'Uberlândia'], [0.65, 0.18, 0.17]),
    'Paraná': (['Curitiba', 'Londrina', 'Maringá'], [0.65, 0.20, 0.15]),
    'Rio Grande do Sul': (['Porto Alegre', 'Caxias do Sul', 'Canoas'], [0.65, 0.20, 0.15]),
    'Bahia': (['Salvador', 'Feira de Santana', 'Vitória da Conquista'], [0.70, 0.18, 0.12]),
    'Pernambuco': (['Recife', 'Olinda', 'Jaboatão dos Guararapes'], [0.65, 0.20, 0.15]),
    'Santa Catarina': (['Florianópolis', 'Joinville', 'Blumenau'], [0.55, 0.27, 0.18]),
    'Goiás': (['Goiânia', 'Aparecida de Goiânia', 'Anápolis'], [0.65, 0.20, 0.15]),
    'Ceará': (['Fortaleza', 'Caucaia', 'Juazeiro do Norte'], [0.70, 0.18, 0.12]),
    'Distrito Federal': (['Brasília', 'Taguatinga', 'Gama'], [0.75, 0.15, 0.10]),
}
states = list(states_cities.keys())

In [7]:
# Probabilidade de estado (maior peso em SP e capitais econômicas)
state_prob = [0.42, 0.08, 0.08, 0.06, 0.05, 0.05, 0.05, 0.06, 0.05, 0.05, 0.05]
state_prob = np.array(state_prob) / np.sum(state_prob)

In [8]:
# ----------------------------
# Departamentos, cargos e probabilidades
# ----------------------------
departments = ['RH', 'TI', 'Vendas', 'Marketing', 'Financeiro', 'Operações', 'Atendimento']
departments_prob = [0.03, 0.22, 0.22, 0.08, 0.08, 0.24, 0.13]

In [9]:
jobtitles = {
    'RH': ['Gerente de RH', 'Coordenador de RH', 'Recrutador', 'Assistente de RH'],
    'TI': ['Gerente de TI', 'Desenvolvedor de Software', 'Administrador de Sistemas', 'Analista de Suporte'],
    'Vendas': ['Gerente de Vendas', 'Consultor de Vendas', 'Especialista de Vendas', 'Representante de Vendas'],
    'Marketing': ['Gerente de Marketing', 'Especialista de SEO', 'Criador de Conteúdo', 'Coordenador de Marketing'],
    'Financeiro': ['Gerente Financeiro', 'Contador', 'Analista Financeiro', 'Assistente de Contas a Pagar'],
    'Operações': ['Gerente de Operações', 'Analista de Operações', 'Coordenador de Logística', 'Especialista de Estoque'],
    'Atendimento': ['Gerente de Atendimento', 'Atendente de Suporte', 'Especialista de Suporte', 'Técnico de Help Desk']
}

In [10]:

jobtitles_prob = {
    'RH': [0.05, 0.15, 0.40, 0.40], # Menos gerentes, mais recrutadores/assistentes.
    'TI': [0.03, 0.60, 0.17, 0.20], # A maior parte desenvolvedores, suporte relevante, poucos gerentes.
    'Vendas': [0.03, 0.15, 0.22, 0.60], # Representantes dominam, poucos gerentes.
    'Marketing': [0.03, 0.25, 0.47, 0.25], # Mais produtores de conteúdo, poucos gerentes.
    'Financeiro': [0.05, 0.35, 0.35, 0.25], # Menos gerentes, balanceado entre analistas e contadores.
    'Operações': [0.03, 0.45, 0.10, 0.42], # Maioria no estoque (operacional), poucos coordenadores e gerentes.
    'Atendimento': [0.03, 0.22, 0.25, 0.50] # Predominância de atendentes, pirâmide bem clara.
}

In [11]:
# Escolaridade
educations = ['Ensino Médio', 'Graduação', 'Mestrado', 'Doutorado']

In [12]:
# Mapeamento cargo -> possíveis escolaridades (valores em português)
education_mapping = {
    'Gerente de RH': ['Mestrado', 'Doutorado'],
    'Coordenador de RH': ['Graduação', 'Mestrado'],
    'Recrutador': ['Ensino Médio', 'Graduação'],
    'Assistente de RH': ['Ensino Médio', 'Graduação'],
    'Gerente de TI': ['Mestrado', 'Doutorado'],
    'Desenvolvedor de Software': ['Graduação', 'Mestrado'],
    'Administrador de Sistemas': ['Graduação', 'Mestrado'],
    'Analista de Suporte': ['Ensino Médio', 'Graduação'],
    'Gerente de Vendas': ['Mestrado', 'Doutorado'],
    'Consultor de Vendas': ['Graduação', 'Mestrado', 'Doutorado'],
    'Especialista de Vendas': ['Graduação', 'Mestrado', 'Doutorado'],
    'Representante de Vendas': ['Graduação'],
    'Gerente de Marketing': ['Graduação', 'Mestrado', 'Doutorado'],
    'Especialista de SEO': ['Ensino Médio', 'Graduação'],
    'Criador de Conteúdo': ['Ensino Médio', 'Graduação'],
    'Coordenador de Marketing': ['Graduação'],
    'Gerente Financeiro': ['Mestrado', 'Doutorado'],
    'Contador': ['Graduação'],
    'Analista Financeiro': ['Graduação', 'Mestrado', 'Doutorado'],
    'Assistente de Contas a Pagar': ['Graduação'],
    'Gerente de Operações': ['Graduação', 'Mestrado'],
    'Analista de Operações': ['Graduação', 'Mestrado'],
    'Coordenador de Logística': ['Graduação'],
    'Especialista de Estoque': ['Ensino Médio', 'Graduação'],
    'Gerente de Atendimento': ['Graduação', 'Mestrado', 'Doutorado'],
    'Atendente de Suporte': ['Ensino Médio', 'Graduação'],
    'Especialista de Suporte': ['Ensino Médio', 'Graduação'],
    'Técnico de Help Desk': ['Ensino Médio', 'Graduação']
}


In [13]:
# ----------------------------
# Datas de admissão (pesos por ano; recência)
# ----------------------------
year_weights_hire = {
    2016: 6,
    2017: 10,
    2018: 12,
    2019: 12,
    2020: 9,
    2021: 10,
    2022: 14,
    2023: 14,
    2024: 8,
    2025: 5
}

In [14]:
def years_between(a: date, b: date) -> int:
    return b.year - a.year - ((b.month, b.day) < (a.month, a.day))

In [15]:
def generate_custom_date(year_weights):
    """Gera uma data dentro de um ano escolhido por pesos e nunca no futuro."""
    year = random.choices(list(year_weights.keys()), weights=list(year_weights.values()))[0]
    start = datetime(year, 1, 1)
    end = datetime(year, 12, 31)
    dt = fake.date_time_between(start_date=start, end_date=end)
    return min(dt, today_dt)  # nunca no futuro

In [16]:
# ----------------------------
# Faixas salariais mensais por cargo (valores aproximados BR; numéricos, sem "R$")
# ----------------------------
def generate_salary(department, job_title):
    # valores em reais/mês (aproximados)
    salary_dict = {
        'RH': {
            'Gerente de RH': np.random.randint(15000, 28000),
            'Coordenador de RH': np.random.randint(9000, 15000),
            'Recrutador': np.random.randint(4000, 9000),
            'Assistente de RH': np.random.randint(3000, 6000)
        },
        'TI': {
            'Gerente de TI': np.random.randint(22000, 35000),
            'Desenvolvedor de Software': np.random.randint(8000, 18000),
            'Administrador de Sistemas': np.random.randint(9000, 17000),
            'Analista de Suporte': np.random.randint(3500, 7500)
        },
        'Vendas': {
            'Gerente de Vendas': np.random.randint(14000, 26000),
            'Consultor de Vendas': np.random.randint(6000, 14000),
            'Especialista de Vendas': np.random.randint(7000, 15000),
            'Representante de Vendas': np.random.randint(4500, 9000)
        },
        'Marketing': {
            'Gerente de Marketing': np.random.randint(13000, 24000),
            'Especialista de SEO': np.random.randint(5000, 11000),
            'Criador de Conteúdo': np.random.randint(3500, 8000),
            'Coordenador de Marketing': np.random.randint(8000, 14000)
        },
        'Financeiro': {
            'Gerente Financeiro': np.random.randint(18000, 32000),
            'Contador': np.random.randint(7000, 14000),
            'Analista Financeiro': np.random.randint(8000, 17000),
            'Assistente de Contas a Pagar': np.random.randint(4000, 7500)
        },
        'Operações': {
            'Gerente de Operações': np.random.randint(14000, 26000),
            'Analista de Operações': np.random.randint(7000, 14000),
            'Coordenador de Logística': np.random.randint(8000, 15000),
            'Especialista de Estoque': np.random.randint(3500, 7000)
        },
        'Atendimento': {
            'Gerente de Atendimento': np.random.randint(12000, 22000),
            'Atendente de Suporte': np.random.randint(2500, 5000),
            'Especialista de Suporte': np.random.randint(3500, 7000),
            'Técnico de Help Desk': np.random.randint(3500, 9000)
        }
    }
    return salary_dict[department][job_title]


In [17]:
# ----------------------------
# Geração de IDs únicos
# ----------------------------
used_ids = set()
def make_employee_id():
    # padrão fictício BR
    while True:
        eid = f"BR-{random.randint(10_000_000, 99_999_999)}"
        if eid not in used_ids:
            used_ids.add(eid)
            return eid

In [18]:
# ----------------------------
# Geração dos dados principais
# ----------------------------
data = []
for _ in range(num_records):
    employee_id = make_employee_id()
    # Gera gênero primeiro
    gender = np.random.choice(['Feminino', 'Masculino'], p=[0.48, 0.52])

# Gera nome compatível com o gênero
    if gender == 'Feminino':
        first_name = fake.first_name_female()
        last_name = fake.last_name()  # sobrenomes neutros em PT-BR
    else:
        first_name = fake.first_name_male()
        last_name = fake.last_name()

    state = np.random.choice(states, p=state_prob)
    cities, city_p = states_cities[state]
    city = np.random.choice(cities, p=city_p)

    hiredate = generate_custom_date(year_weights_hire)

    department = np.random.choice(departments, p=departments_prob)
    job_title = np.random.choice(jobtitles[department], p=jobtitles_prob[department])

    education_level = np.random.choice(education_mapping[job_title])

    performance_rating = np.random.choice(
        ['Excelente', 'Bom', 'Satisfatório', 'Precisa Melhorar'],
        p=[0.14, 0.50, 0.28, 0.08]
    )
    # overtime pode variar por departamento (ex.: maior em Operações/Atendimento)
    overtime_probs = {
        'RH': 0.18, 'TI': 0.28, 'Vendas': 0.22, 'Marketing': 0.20,
        'Financeiro': 0.24, 'Operações': 0.38, 'Atendimento': 0.34
    }
    overtime = np.random.choice(['Sim', 'Não'], p=[overtime_probs[department], 1 - overtime_probs[department]])

    salary = generate_salary(department, job_title)

    data.append([
        employee_id, first_name, last_name, gender, state, city, hiredate,
        department, job_title, education_level, salary, performance_rating, overtime
    ])

columns = [
    'employee_id', 'first_name', 'last_name', 'gender',
    'state', 'city', 'hiredate',
    'department', 'job_title', 'education_level',
    'salary', 'performance_rating', 'overtime'
]
df = pd.DataFrame(data, columns=columns)

In [19]:
# ----------------------------
# Data de nascimento (regras simples de coerência)
# ----------------------------
def _clamp_day(year: int, month: int, day: int) -> int:
    """Devolve um dia válido (<= último dia do mês/ano)."""
    return min(day, monthrange(year, month)[1])

def generate_birthdate(row):
    # idade mínima na admissão
    min_age_at_hire = 25 if 'Gerente' in row['job_title'] else 18

    hire_d = pd.to_datetime(row['hiredate']).date()
    # data máxima permitida p/ nascer (para ter a idade mínima na admissão)
    y = hire_d.year - min_age_at_hire
    m = hire_d.month
    d = _clamp_day(y, m, hire_d.day)  # evita 29/02 em ano não bissexto e 31 em meses de 30
    latest_birth_allowed = date(y, m, d)

    # distribuição/heurística de idades (como você já tinha)
    age_distribution = {'under_25': 0.10, '25_34': 0.28, '35_44': 0.32, '45_54': 0.20, 'over_55': 0.10}
    age_group = np.random.choice(list(age_distribution.keys()), p=list(age_distribution.values()))

    if 'Gerente' in row['job_title']:
        target_age_today = np.random.randint(30, 66)
    elif row['education_level'] == 'Doutorado':
        target_age_today = np.random.randint(27, 66)
    elif age_group == 'under_25':
        target_age_today = np.random.randint(20, 25)
    elif age_group == '25_34':
        target_age_today = np.random.randint(25, 35)
    elif age_group == '35_44':
        target_age_today = np.random.randint(35, 45)
    elif age_group == '45_54':
        target_age_today = np.random.randint(45, 55)
    else:
        target_age_today = np.random.randint(56, 66)

    # converte "idade alvo hoje" em data de nascimento, com clamp de dia
    today_d = pd.Timestamp('today').date()
    yt = today_d.year - target_age_today
    mt = today_d.month
    dt = _clamp_day(yt, mt, today_d.day)
    tentative_birth = date(yt, mt, dt)

    # garante a idade mínima na admissão
    if tentative_birth > latest_birth_allowed:
        tentative_birth = latest_birth_allowed

    return tentative_birth

In [20]:
df['birthdate'] = df.apply(generate_birthdate, axis=1)

In [21]:
# ----------------------------
# Demissões (termdate) com coerência de ≥ 6 meses de casa
# ----------------------------
termination_percentage = 0.115  # ~11,5%

year_weights_term = {
    2017: 7,
    2018: 10,
    2019: 11,
    2020: 15,   # pico pandêmico
    2021: 13,
    2022: 14,
    2023: 12,
    2024: 10,
    2025: 8
}

In [22]:
# elegíveis: hiredate + 180d <= hoje
df['hiredate'] = pd.to_datetime(df['hiredate']).dt.date
elig_mask = pd.to_datetime(df['hiredate']) + pd.to_timedelta(180, unit='D') <= pd.Timestamp(today_d)
eligible_idx = df.index[elig_mask]

total_terminated = int(round(len(df) * termination_percentage))
total_terminated = min(total_terminated, len(eligible_idx))  # segurança

In [23]:
# amostra de índices elegíveis
terminated_indices = np.random.choice(eligible_idx, size=total_terminated, replace=False)

# sorteia anos de demissão pelos pesos
years = list(year_weights_term.keys())
weights = np.array(list(year_weights_term.values()), dtype=float)
weights = weights / weights.sum()
termination_years = np.random.choice(years, size=total_terminated, p=weights, replace=True)

In [24]:
df['termdate'] = pd.NaT  # NaT para ativos


In [25]:
for idx, year in zip(terminated_indices, termination_years):
    # janela aleatória dentro do ano de demissão
    raw_term = datetime(year, 1, 1) + timedelta(days=random.randint(0, 364))
    min_term = pd.Timestamp(df.at[idx, 'hiredate']) + pd.Timedelta(days=180)
    term_final = max(pd.Timestamp(raw_term), min_term)

    # evita futuro
    if term_final > pd.Timestamp(today_d):
        term_final = pd.Timestamp(today_d)

    df.at[idx, 'termdate'] = term_final


In [26]:
# ----------------------------
# Ajuste salarial por escolaridade e tenure (anos de casa)
# ----------------------------
# multiplicadores por escolaridade (neutro por gênero)
education_multiplier = {
    'Ensino Médio': 1.00,
    'Graduação': 1.08,
    'Mestrado': 1.16,
    'Doutorado': 1.28
}

In [27]:
def calculate_adjusted_salary(row):
    base_salary = row['salary']
    mult = education_multiplier.get(row['education_level'], 1.0)
    adjusted = base_salary * mult

    tenure_years = years_between(row['hiredate'], today_d)

    # incremento por tenure: 0,3% a 0,7% ao ano de casa
    tenure_increment = 1 + np.random.uniform(0.003, 0.007) * max(0, tenure_years)
    adjusted *= tenure_increment

    return int(round(max(adjusted, base_salary)))

In [28]:
df['salary'] = df.apply(calculate_adjusted_salary, axis=1)

In [29]:
# ----------------------------
# Conversões finais e colunas úteis para o BI
# ----------------------------
df['birthdate'] = pd.to_datetime(df['birthdate']).dt.date
df['termdate'] = pd.to_datetime(df['termdate']).dt.date

df['is_active'] = df['termdate'].isna()
df['tenure_years'] = df.apply(lambda r: years_between(r['hiredate'], today_d), axis=1)
df['salary_annual_13x'] = df['salary'] * 13  # anual considerando 13º
df['seniority_band'] = pd.cut(
    df['tenure_years'],
    bins=[-1, 1, 3, 6, 100],
    labels=['0–1', '1–3', '3–6', '6+']
)


In [30]:
# mapa simples de estado -> UF
state_to_uf = {
    'São Paulo': 'SP', 'Rio de Janeiro': 'RJ', 'Minas Gerais': 'MG', 'Paraná': 'PR',
    'Rio Grande do Sul': 'RS', 'Bahia': 'BA', 'Pernambuco': 'PE', 'Santa Catarina': 'SC',
    'Goiás': 'GO', 'Ceará': 'CE', 'Distrito Federal': 'DF'
}
df['state_code'] = df['state'].map(state_to_uf)

In [31]:
# ----------------------------
# Amostra e exportação
# ----------------------------
print(df.head())

   employee_id   first_name last_name     gender             state       city  \
0  BR-95822412       Brenda     Alves   Feminino  Distrito Federal   Brasília   
1  BR-46913810      Joaquim    Câmara  Masculino         São Paulo   Campinas   
2  BR-28728463         Cauã     Rocha  Masculino            Paraná    Maringá   
3  BR-83197857         Yago    Borges  Masculino            Paraná   Curitiba   
4  BR-66629388  Ana Beatriz     Alves   Feminino         São Paulo  São Paulo   

     hiredate  department                  job_title education_level  salary  \
0  2017-09-27  Financeiro                   Contador       Graduação    8293   
1  2018-02-20          TI  Desenvolvedor de Software        Mestrado   17042   
2  2023-09-27          TI  Desenvolvedor de Software       Graduação   13973   
3  2017-08-03          TI  Desenvolvedor de Software        Mestrado   10116   
4  2016-02-04          TI        Analista de Suporte    Ensino Médio    7124   

  performance_rating overtime   

In [32]:
mask = df['termdate'].notna()

ok_term_6m = (
    pd.to_datetime(df.loc[mask, 'termdate']) >=
    pd.to_datetime(df.loc[mask, 'hiredate']) + pd.Timedelta(days=180)
).all()

ok_term_6m

np.True_

In [33]:
violations = df.loc[
    mask & (
        pd.to_datetime(df['termdate']) <
        pd.to_datetime(df['hiredate']) + pd.Timedelta(days=180)
    ),
    ['employee_id', 'hiredate', 'termdate', 'department', 'job_title']
]
violations.head(10), len(violations)

(Empty DataFrame
 Columns: [employee_id, hiredate, termdate, department, job_title]
 Index: [],
 0)

In [34]:
check_min_age = (
    (pd.to_datetime(df['hiredate']) - pd.to_datetime(df['birthdate'])).dt.days >= 18*365
).all()
check_min_age_ger = (
    ((~df['job_title'].str.contains('Gerente')) |
     ((pd.to_datetime(df['hiredate']) - pd.to_datetime(df['birthdate'])).dt.days >= 25*365))
).all()
print({'idade_minima_18': check_min_age, 'idade_minima_gerente_25': check_min_age_ger})

{'idade_minima_18': np.True_, 'idade_minima_gerente_25': np.True_}


In [35]:
# Guards finais
age_at_hire_days = (pd.to_datetime(df['hiredate']) - pd.to_datetime(df['birthdate'])).dt.days
assert (age_at_hire_days >= 18*365).all()
assert (
    (~df['job_title'].str.contains('Gerente')) | (age_at_hire_days >= 25*365)
).all()

In [36]:
# nome de arquivo
df.to_csv('RecursosHumanos_BR.csv', index=False)